# [286. Walls and Gates](https://leetcode.com/problems/walls-and-gates/description/?envType=company&envId=doordash&favoriteSlug=doordash-six-months)

You are given an `m x n` grid `rooms`initialized with these three possible values.

- `-1`A wall or an obstacle.
- `0` A gate.
- `INF` Infinity means an empty room. We use the value `2^31 - 1 = 2147483647` to represent `INF` as you may assume that the distance to a gate is less than `2147483647`.

Fill each empty room with the distance to its nearest gate. If it is impossible to reach a gate, it should be filled with `INF`.

**Example 1:** 
<img alt="" src="https://assets.leetcode.com/uploads/2021/01/03/grid.jpg" style="width: 500px; height: 223px;">

```
Input: rooms = [[2147483647,-1,0,2147483647],[2147483647,2147483647,2147483647,-1],[2147483647,-1,2147483647,-1],[0,-1,2147483647,2147483647]]
Output: [[3,-1,0,1],[2,2,1,-1],[1,-1,2,-1],[0,-1,3,4]]
```

**Example 2:** 

```
Input: rooms = [[-1]]
Output: [[-1]]
```

**Constraints:** 

- `m == rooms.length`
- `n == rooms[i].length`
- `1 <= m, n <= 250`
- `rooms[i][j]` is `-1`, `0`, or `2^31 - 1`.

In [ ]:
from collections import deque

class Solution:
    def wallsAndGates(self, rooms: list[list[int]]) -> None:
        """
        Do not return anything, modify rooms in-place instead.
        """

        # Multi-source BFS approach

        queue = deque()
        INF = 2147483647

        for i in range(len(rooms)):
            for j in range(len(rooms[0])):
                if rooms[i][j] == 0:
                    queue.append(((i,j), 0))
        
        neighbors = [(-1,0),(1,0),(0,1),(0,-1)]
        
        while queue:
            (x, y), dist = queue.popleft()

            for dx, dy in neighbors:
                nx, ny = x + dx, y + dy

                # invalid: outside board, wall, or already marked
                if 0 <= nx < len(rooms) and 0 <= ny < len(rooms[0]) and rooms[nx][ny] == INF:
                    rooms[nx][ny] = dist + 1

                    queue.append(((nx, ny), dist + 1))


## TIME & SPACE COMPLEXITY

Time complexity is O(m * n) , but technically m*n represents the number of nodes. Each node only has 4 adjacencies / edges, which is constant. In this algo we only visit each cell once

Space complexity is O(m * n). At worst we could have every single node / cell in the queue at once

## EXTENSIONS

Do walls & gates, but in the case of equidistant gates, you need to mark the source gate of that cell as the gate farthest to the left.

For example, if gate (1, 3) & (1, 1) are both equidistant to a cell, we need to mark (1, 1) as the source gate.

In [ ]:
from collections import deque

class Solution:
    def wallsAndGates(self, rooms: list[list[int]]) -> None:
        """
        Do not return anything, modify rooms in-place instead.
        """

        # Multi-source BFS approach
        queue = deque()
        INF = 2147483647

        for i in range(len(rooms)):
            for j in range(len(rooms[0])):
                if rooms[i][j] == 0:
                    queue.append(((i,j), 0, (i,j)))
        
        neighbors = [(-1,0),(1,0),(0,1),(0,-1)]

        source_gates = [[None for _ in range(len(rooms[0]))] for _ in range(len(rooms))]
        
        while queue:
            (x, y), dist, (i, j) = queue.popleft()

            for dx, dy in neighbors:
                nx, ny = x + dx, y + dy

                # invalid: outside board, wall, or already marked
                if 0 <= nx < len(rooms) and 0 <= ny < len(rooms[0]) and rooms[nx][ny] != -1:
                    if rooms[nx][ny] == INF: # never been marked before
                        rooms[nx][ny] = dist + 1
                        queue.append(((nx, ny), dist + 1, (i, j)))
                        source_gates[nx][ny] = (i, j)
                    elif rooms[nx][ny] == dist + 1: 
                        # distance from this source to neighbor is same as already marked
                        if j < source_gates[nx][ny][1]:
                            source_gates[nx][ny] = (i, j)
                            queue.append(((nx, ny), dist + 1, (i, j)))
                            # Have to re-add this new source to the queue to propagate things

        

## TIME & SPACE COMPLEXITY

Time complexity worsens in this scenario because we need to propagate the source nodes which can cause re-processing of various cells to find the leftmost equidistant gate. It's essentially the same as running BFS from each gate. Remember that since each node has constant number of adjacencies (4), our time complexity was only O(V) = O(m * n) . With this extension that would become O(G * m * n)

Space complexity is also O(G * m * n). At worst we could have every single node / cell in the queue at once, with every gate as a source for each node